In [1]:
!pip install pyspark==3.4.3 py4j==0.10.9.7 # supported version change

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.4/311.4 MB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.3-py2.py3-none-any.whl size=311885495 sha256=5b0830738ad251e8c651fecc07293c29fce62c51a14887d347e43888e68c0a8c
  Stored in directory: /root/.cache/pip/wheels/ea/a9/64/3713eb2c5048c18bae2778b013e5fc74203f5c22d4640fb776
Successfully built pyspark


In [2]:
!wget https://s3-geospatial.s3-us-west-2.amazonaws.com/Marvel_names.txt -P ./data

--2024-07-08 06:20:59--  https://s3-geospatial.s3-us-west-2.amazonaws.com/Marvel_names.txt
Resolving s3-geospatial.s3-us-west-2.amazonaws.com (s3-geospatial.s3-us-west-2.amazonaws.com)... 52.92.179.122, 3.5.84.177, 52.92.133.170, ...
Connecting to s3-geospatial.s3-us-west-2.amazonaws.com (s3-geospatial.s3-us-west-2.amazonaws.com)|52.92.179.122|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 351848 (344K) [text/plain]
Saving to: ‘./data/Marvel_names.txt’

Marvel_names.txt    100%[===================>] 343.60K  1.36MB/s    in 0.2s    

2024-07-08 06:20:59 (1.36 MB/s) - ‘./data/Marvel_names.txt’ saved [351848/351848]



In [3]:
!head -5 ./data/Marvel_names.txt

1 "24-HOUR MAN/EMMANUEL"
2 "3-D MAN/CHARLES CHAN"
3 "4-D MAN/MERCURIO"
4 "8-BALL/"
5 "A"


In [4]:
!wget https://s3-geospatial.s3-us-west-2.amazonaws.com/Marvel_graph.txt -P ./data

--2024-07-08 06:21:00--  https://s3-geospatial.s3-us-west-2.amazonaws.com/Marvel_graph.txt
Resolving s3-geospatial.s3-us-west-2.amazonaws.com (s3-geospatial.s3-us-west-2.amazonaws.com)... 52.92.200.106, 52.92.201.146, 52.92.147.234, ...
Connecting to s3-geospatial.s3-us-west-2.amazonaws.com (s3-geospatial.s3-us-west-2.amazonaws.com)|52.92.200.106|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1673543 (1.6M) [text/plain]
Saving to: ‘./data/Marvel_graph.txt’

Marvel_graph.txt    100%[===================>]   1.60M  4.27MB/s    in 0.4s    

2024-07-08 06:21:00 (4.27 MB/s) - ‘./data/Marvel_graph.txt’ saved [1673543/1673543]



In [5]:
!head -5 ./data/Marvel_graph.txt

5988 748 1722 3752 4655 5743 1872 3413 5527 6368 6085 4319 4728 1636 2397 3364 4001 1614 1819 1585 732 2660 3952 2507 3891 2070 2239 2602 612 1352 5447 4548 1596 5488 1605 5517 11 479 2554 2043 17 865 4292 6312 473 534 1479 6375 4456 
5989 4080 4264 4446 3779 2430 2297 6169 3530 3272 4282 6432 2548 4140 185 105 3878 2429 1334 4595 2767 3956 3877 4776 4946 3407 128 269 5775 5121 481 5516 4758 4053 1044 1602 3889 1535 6038 533 3986 
5982 217 595 1194 3308 2940 1815 794 1503 5197 859 5096 6039 2664 651 2244 528 284 1449 1097 1172 1092 108 3405 5204 387 4607 4545 3705 4930 1805 4712 4404 247 4754 4427 1845 536 5795 5978 533 3984 6056 
5983 1165 3836 4361 1282 716 4289 4646 6300 5084 2397 4454 1913 5861 5485 
5980 2731 3712 1587 6084 2472 2546 6313 875 859 323 2664 1469 522 2506 2919 2423 3624 5736 5046 1787 5776 3245 3840 2399 


In [6]:
!ls -tl

total 8
drwxr-xr-x 2 root root 4096 Jul  8 06:21 data
drwxr-xr-x 1 root root 4096 Jul  3 13:21 sample_data


In [7]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
        .master("local[*]") \
        .appName("PySpark Finding Super-hero") \
        .getOrCreate()

In [8]:
import pyspark.sql.functions as F
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

In [9]:
# schema 작성 (Marvel names)

marvel_name_schema = StructType([
      StructField("id", StringType(), True),
      StructField("name", StringType(), True)
])

In [10]:
names = spark.read.schema(marvel_name_schema).option("sep", " ").csv("./data/Marvel_names.txt")

In [11]:
names.show(5)

+---+--------------------+
| id|                name|
+---+--------------------+
|  1|24-HOUR MAN/EMMANUEL|
|  2|3-D MAN/CHARLES CHAN|
|  3|    4-D MAN/MERCURIO|
|  4|             8-BALL/|
|  5|                   A|
+---+--------------------+
only showing top 5 rows



In [12]:
lines = spark.read.text("./data/Marvel_graph.txt")

In [13]:
lines.show(5)

+--------------------+
|               value|
+--------------------+
|5988 748 1722 375...|
|5989 4080 4264 44...|
|5982 217 595 1194...|
|5983 1165 3836 43...|
|5980 2731 3712 15...|
+--------------------+
only showing top 5 rows



In [14]:
# lines의 스키마에서 value 필드를 super hero ID("id")와 카운트("connections")로 변경
# 전략 - 공백 중심으로 split해서 처음 값이 super hero ID이고 뒤의 값이 총 수가 됨

In [15]:
id_connections = lines.withColumn("id", F.split(F.trim(F.col("value")), " ")[0]) \
                      .withColumn("connections", F.size(F.split(F.trim(F.col("value")), " ")) - 1)

In [16]:
id_connections.select("id", "connections").show(5)

+----+-----------+
|  id|connections|
+----+-----------+
|5988|         48|
|5989|         40|
|5982|         42|
|5983|         14|
|5980|         24|
+----+-----------+
only showing top 5 rows



In [17]:
connections = id_connections.groupBy("id").agg(F.sum("connections").alias("connections"))

In [18]:
connections.show()

+----+-----------+
|  id|connections|
+----+-----------+
| 691|          6|
|1159|         11|
|3959|        142|
|1572|         35|
|2294|         14|
|1090|          4|
|3606|        171|
|3414|          7|
| 296|         17|
|4821|         16|
|2162|         41|
|1436|          9|
|1512|         11|
|6194|         14|
|6240|         11|
| 829|         37|
|2136|          6|
|5645|         20|
|2069|        263|
| 467|          0|
+----+-----------+
only showing top 20 rows



In [19]:
mostPopular = connections.sort(F.col("connections").desc()).first()

In [20]:
mostPopular

Row(id='859', connections=1933)

In [21]:
mostPopularName = names.filter(F.col("id") == mostPopular[0]).select("name").first()

In [22]:
print(mostPopularName[0] + "가 가장 인기 있는 슈퍼 히어로로 총 " + str(mostPopular[1]) + "번 다른 히어로들과 출연했습니다.")

CAPTAIN AMERICA가 가장 인기 있는 슈퍼 히어로로 총 1933번 다른 히어로들과 출연했습니다.


### 위 후반부 문제 풀이를 Spark SQL로 해보자

In [23]:
id_connections.createOrReplaceTempView("id_connections")
names.createOrReplaceTempView("names")

In [24]:
spark.sql("""
  SELECT c.id AS conn_id, n.name AS name, SUM(c.connections) AS connections
  FROM id_connections AS c
  INNER JOIN names AS n
    ON c.id = n.id
  GROUP BY conn_id, name
  ORDER BY connections DESC;
""").show()

+-------+--------------------+-----------+
|conn_id|                name|connections|
+-------+--------------------+-----------+
|    859|     CAPTAIN AMERICA|       1933|
|   5306|SPIDER-MAN/PETER PAR|       1741|
|   2664|IRON MAN/TONY STARK |       1528|
|   5716|THING/BENJAMIN J. GR|       1426|
|   6306|    WOLVERINE/LOGAN |       1394|
|   3805|MR. FANTASTIC/REED R|       1386|
|   2557|HUMAN TORCH/JOHNNY S|       1371|
|   4898|SCARLET WITCH/WANDA |       1345|
|   5736|THOR/DR. DONALD BLAK|       1289|
|    403|BEAST/HENRY &HANK& P|       1280|
|   6066|             VISION |       1263|
|   2650|INVISIBLE WOMAN/SUE |       1244|
|   2399|                HAWK|       1176|
|   1289|CYCLOPS/SCOTT SUMMER|       1104|
|   5467|STORM/ORORO MUNROE S|       1095|
|    133|ANGEL/WARREN KENNETH|       1094|
|   6148|WASP/JANET VAN DYNE |       1093|
|    154|ANT-MAN/DR. HENRY J.|       1092|
|   5046|SHE-HULK/JENNIFER WA|       1080|
|   1602|DR. STRANGE/STEPHEN |       1079|
+-------+--

### 이번에는 UDF를 사용해서 히어로 ID Lookup을 구현해보자

In [25]:
def returnName(id: str) -> str:
  return id2Names.get(id) # closure 에서 lookup하는 경우 (task 기반 직렬화 가능 -> 성능 저하)
  # return broadcast_id2Names.value.get(id) # broadcast variable 기반 수행 (worker 별로 직렬화 가능 -> 성능이 상대적으로 양호)

In [26]:
id2Names = names.rdd.collectAsMap()

In [27]:
len(id2Names)

19428

In [28]:
for idx, id in enumerate(id2Names):
  if idx >= 10:
    break
  print(id, id2Names[id])

1 24-HOUR MAN/EMMANUEL
2 3-D MAN/CHARLES CHAN
3 4-D MAN/MERCURIO
4 8-BALL/
5 A
6 A'YIN
7 ABBOTT, JACK
8 ABCISSA
9 ABEL
10 ABOMINATION/EMIL BLO


In [29]:
broadcast_id2Names = spark.sparkContext.broadcast(id2Names)

In [30]:
broadcast_id2Names.value.get('859')

'CAPTAIN AMERICA'

In [31]:
spark.udf.register("returnName", returnName, StringType())

<function __main__.returnName(id: str) -> str>

In [32]:
id_connections.withColumn("name", F.expr("returnName(id)")).show()

+--------------------+----+-----------+--------------------+
|               value|  id|connections|                name|
+--------------------+----+-----------+--------------------+
|5988 748 1722 375...|5988|         48|VALKYRIE II | MUTANT|
|5989 4080 4264 44...|5989|         40|         VALKYRIE IV|
|5982 217 595 1194...|5982|         42|VAGABOND/PRISCILLA L|
|5983 1165 3836 43...|5983|         14|              VAGUE/|
|5980 2731 3712 15...|5980|         24|         UTGARD-LOKI|
|5981 3569 5353 40...|5981|         17|              VACUUM|
|5986 2658 3712 26...|5986|        142|             VALINOR|
|5987 2614 5716 17...|5987|         81|              VALKIN|
|5984 590 4898 745...|5984|         41|              VAKUME|
|5985 3233 2254 21...|5985|         19|             VALERIA|
|6294 4898 1127 32...|6294|         13|WOLF SPIRIT/OWAYODAT|
|270 2658 3003 380...| 270|         42|AUGUST PERSONAGE IN |
|271 4935 5716 430...| 271|          9|      AUNTIE FREEZE/|
|272 2717 4363 408...| 2